# File Integrity

The basic question is: "Has the file changed since the last time you used it?"

## Hash (Browns)

The layman definition of a hash: A fixed-length, scrambled string that uniquely identifies "a thing".

The layman definition of a hashing function: A function that transforms "a thing" into a hash.

## `hashlib`

Provides a library of hashing functions for hashing objects, strings, etc.

In [2]:
from hashlib import sha256, md5

m = sha256()
m.update('hello'.encode('utf-8'))
m.hexdigest()

'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

## Properties of hashes & `hashlib` functions

Hashes of the same "thing" should yield the same hash value.

In [3]:
m2 = sha256()
m2.update('hello'.encode('utf-8'))
m2.hexdigest()

'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

Similar-looking but different strings will yield different hashes.

In [4]:
m3 = sha256()
m3.update('héllo'.encode('utf-8'))
m3.hexdigest()

'3c48591d8d098a4538f5e013dfcf406e948eac4d3277b10bf614e295d6068179'

Using a different hashing algorithm will yield a different hash.

In [5]:
n = md5()
n.update('hello'.encode('utf-8'))
n.hexdigest()

'5d41402abc4b2a76b9719d911017c592'

Hashing functions don't work on all objects.

In [6]:
try:
    o = sha256()
    o.update(3)
except TypeError:
    print('Numbers cannot be hashed')

Numbers cannot be hashed


In [7]:
try:
    o = sha256()
    o.update('Hello world!')
except TypeError:
    print('Strings cannot be hashed without encoding.')

Strings cannot be hashed without encoding.


In [8]:
try:
    o = sha256()
    o.update('Hello world!'.encode('utf-8'))
    print(o.hexdigest())
except TypeError:
    print('Strings must be encoded first.')

c0535e4be2b79ffd93291305436bf889314e4a3faec05ecffcbb7df31ad9e51a


## Checking for changes in data file

Multiple approaches possible:

- Check every cell against a "master" copy, assuming you have one. (**inefficient, but good for pinpointing tampered cells**)
- Check every row against a hash of that row. (**somewhat inefficient, but good for practice, and good for pinpointing tampered rows**)
- Check hash of a file. (**most efficient way**)

## Exercise Part 1

- Write a function that hashes strings and returns the digest, and add it to `datafuncs.py`. It should wrap the SHA256 algorithm.

In [9]:
def hash_string(string):
    """
    Convenience wrapper that returns the hash of a string.
    """
    string = string.encode('utf-8')
    return sha256(string).hexdigest()
    
hash_string('hello')

'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

## Exercise Part 2

- Use `pandas` to open the data file, `data/Divvy_Stations_2013.csv` as the variable name `df`. 
- Create a new DataFrame called `hashes`.
- Create a new column in `hashes` called `concat`, which is each column of data from `df` converted to strings and concatenated into a contiguous string.
- Create a new column in `hashes` called `hash`, which is the computed the hash of each row of the contiguous strings.
- Delete the `concat` column from `hashes`.
- Save the hashes to disk as the file `hashes.csv`.

In [10]:
import pandas as pd

df = pd.read_csv('data/Divvy_Stations_2013.csv')
hashes = pd.DataFrame()  # don't modify the original dataframe.
hashes['concat'] = df.apply(lambda x: ''.join(str(x[col]) for col in df.columns), axis=1)
hashes['hash'] = hashes['concat'].apply(lambda x: hash_string(x))
del hashes['concat']
hashes.head()

,hash
0,1aaf1b6343c89f1c7175fbad7f3ca1011fd9454da1169d...
1,8a5215386187efb140c0050cea72d91b2a661d196d3176...
2,8523ece8190a5049948d9219fe69a2dc48909da896a6c3...
3,48983a013b5ad6c19cec8b3d02ec94139372e23fc1d1e0...
4,6d3979c8ee2737e7cdce6dbe937d914688c973cb2d0ef5...


## Exercise Part 3

- Now, wrap this in a function too!

In [11]:
def hash_data(dataframe):
    hashes = pd.DataFrame()  # don't modify the original
    hashes['concat'] = df.apply(lambda x: ''.join(str(x[col]) for col in df.columns), axis=1)
    hashes['hash'] = hashes['concat'].apply(lambda x: hash_string(x))
    del hashes['concat']
    return hashes

hash_data(df)['hash'].head()

0    1aaf1b6343c89f1c7175fbad7f3ca1011fd9454da1169d...
1    8a5215386187efb140c0050cea72d91b2a661d196d3176...
2    8523ece8190a5049948d9219fe69a2dc48909da896a6c3...
3    48983a013b5ad6c19cec8b3d02ec94139372e23fc1d1e0...
4    6d3979c8ee2737e7cdce6dbe937d914688c973cb2d0ef5...
Name: hash, dtype: object

## Hash of a file

It is possible to check the hash of a file. Let's add an existing implementation found online to our toolkit, `datafuncs.py`.

(All credit to StackOverflow community: http://stackoverflow.com/questions/3431825/generating-an-md5-checksum-of-a-file)

In [12]:
def hash_file(fname):
    filehash = sha256()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            filehash.update(chunk)
    return filehash.hexdigest()

In [13]:
hash_file('data/Divvy_Stations_2013.csv')

'c861005089beb7f09e26a5b7afa09843a0ac1ca98fe9c36ac0510a58b21da40d'

In [14]:
hash_file('data/Divvy_Stations_2013_corrupt.csv')

'880ba1ef2e38e4c35df4b2cd745529797f08fb24048dea0600e8174518a99869'

## Exercise

- Create a CSV file from a `pandas.DataFrame()` (or create a [`tinydb`](http://tinydb.readthedocs.io/en/latest/) database) to store the MD5 hashes of each file in the `data/` directory. Place this in the directory called `data_integrity/`. Be sure to record:
    - File name.
    - Hash.
    - Date and time on which hash was computed.

In [19]:
import os
from tinydb import TinyDB, Query
from datetime import datetime, date

db = TinyDB('data_integrity/hashes.db')

data_prefix = 'data'
for f in os.listdir(data_prefix):
    filehash = hash_file(f'{data_prefix}/{f}')
    record = dict()
    record['filename'] = f'{data_prefix}/{f}'
    record['hash'] = filehash
    record['datetime_hashed'] = datetime.today().isoformat()
    db.insert(record)

## Exercise

- Write a test that checks that the hash is the value that was recorded.
- If you used a `tinydb` database, then check the API docs [here][tinydb] for more information on how to query for a particular record.

[tinydb]: http://tinydb.readthedocs.io/en/latest/

In [24]:
def test_file_hashes():
    db = TinyDB('data_integrity/hashes.db')
    
    for f in os.listdir(data_prefix):
        filename = f'{data_prefix}/{f}'
        filehash = hash_file(filename)
        Rec = Query()
        latest_record = db.search(Rec.filename == filename)[-1]
        assert latest_record['hash'] == filehash

test_file_hashes()